# Grabs Raw Coefficients from kassiesa.net

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
PATH = 'c:\\Users\\calvin\\Documents\\GitHub\\springboard\\champions_league_luck'
starturl = 'https://kassiesa.net/uefa/data/method2/trank2000.html'
kassiesaPath = PATH + '/data/raw/kassiesa/'

In [3]:
games = pd.read_csv(f"{PATH}\\data\\interim\\games.csv", index_col=0, dtype={'rnd':'category', 'day':'category', 'season':'category'}, parse_dates=['date'])
games.head()

,season,rnd,date,day,notes,wk,h_team,h_country,h_score,a_team,a_country,a_score,h_pens,a_pens
993,2000-2001,First group stage,2000-09-12,Tue,NaN,1,Sporting CP,pt,2,Real Madrid,es,2,0,0
994,2000-2001,First group stage,2000-09-12,Tue,NaN,1,Lyon,fr,3,Heerenveen,nl,1,0,0
995,2000-2001,First group stage,2000-09-12,Tue,NaN,1,Galatasaray,tr,3,Monaco,fr,2,0,0
996,2000-2001,First group stage,2000-09-12,Tue,NaN,1,Rangers,sco,5,Sturm Graz,at,0,0,0
997,2000-2001,First group stage,2000-09-12,Tue,NaN,1,Sparta Prague,cz,0,Arsenal,eng,1,0,0


In [4]:
games[games['rnd'] == "Final"].sort_values(by='date')

,season,rnd,date,day,notes,wk,h_team,h_country,h_score,a_team,a_country,a_score,h_pens,a_pens
1149,2000-2001,Final,2001-05-23,Wed,Required Extra Time,0,Bayern Munich,de,1,Valencia,es,1,0,0
1306,2001-2002,Final,2002-05-15,Wed,NaN,0,Leverkusen,de,1,Real Madrid,es,2,0,0
1463,2002-2003,Final,2003-05-28,Wed,Milan won on penalty kicks following extra time,0,Juventus,it,0,Milan,it,0,2,3
1588,2003-2004,Final,2004-05-26,Wed,NaN,0,Monaco,fr,0,Porto,pt,3,0,0
1713,2004-2005,Final,2005-05-25,Wed,Liverpool won on penalty kicks following extra...,0,Milan,it,3,Liverpool,eng,3,2,3
1838,2005-2006,Final,2006-05-17,Wed,NaN,0,Barcelona,es,2,Arsenal,eng,1,0,0
1963,2006-2007,Final,2007-05-23,Wed,NaN,0,Milan,it,2,Liverpool,eng,1,0,0
2088,2007-2008,Final,2008-05-21,Wed,Manchester Utd won on penalty kicks following ...,0,Manchester Utd,eng,1,Chelsea,eng,1,6,5
2213,2008-2009,Final,2009-05-27,Wed,NaN,0,Barcelona,es,2,Manchester Utd,eng,0,0,0
2338,2009-2010,Final,2010-05-22,Sat,NaN,0,Bayern Munich,de,0,Inter,it,2,0,0


## Choosing the Data
I chose the club ranking data from kassiesa.net. Seems very well researched and it's easy to get every year that I need.
I will need to quickly scrape and format first

In [5]:
# Let's see if we can get a table from pandas method
test = pd.read_html('https://kassiesa.net/uefa/data/method2/trank2000.html')
test[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,95/96,96/97,97/98,98/99,99/00,ranking
0,1.0,Juventus,Ita,22.7500,27.071,23.1425,20.0000,17.0000,109.963
1,2.0,FC Barcelona,Esp,22.5000,22.800,9.2140,12.0000,37.2855,103.799
2,3.0,Bayern München,Ger,28.4165,6.571,16.5000,22.1785,29.5355,103.201
3,4.0,Real Madrid,Esp,16.5000,5.800,25.2140,16.0000,36.2855,99.799
4,5.0,Lazio,Ita,8.7500,10.071,26.1425,22.0000,28.0000,94.963
...,...,...,...,...,...,...,...,...,...
272,NaN,Azerbaijan,NaN,0.1250,0.333,NaN,NaN,0.1665,0.624
273,NaN,Liechtenstein,NaN,NaN,0.500,NaN,NaN,NaN,0.500
274,NaN,Albania,NaN,0.1665,NaN,NaN,0.1665,0.0830,0.416
275,NaN,Bosnia-Herzegovina,NaN,NaN,NaN,NaN,0.2500,NaN,0.250


In [6]:
headers = {'User-Agent': 'My User Agent 1.0'}

In [7]:
def nextpage(url: str):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    for b in soup.find_all('b'):
        if '>' in b.text:
            nexturl = urljoin(r.url, b.a['href'])
            with open(f"{kassiesaPath}{url[-14:]}", 'w', encoding='utf8') as f:
                f.write(r.text)
            print(nexturl)
            nextpage(nexturl)


In [8]:
nextpage(starturl)

https://kassiesa.net/uefa/data/method2/trank2001.html
https://kassiesa.net/uefa/data/method2/trank2002.html
https://kassiesa.net/uefa/data/method2/trank2003.html
https://kassiesa.net/uefa/data/method3/trank2004.html
https://kassiesa.net/uefa/data/method3/trank2005.html
https://kassiesa.net/uefa/data/method3/trank2006.html
https://kassiesa.net/uefa/data/method3/trank2007.html
https://kassiesa.net/uefa/data/method3/trank2008.html
https://kassiesa.net/uefa/data/method4/trank2009.html
https://kassiesa.net/uefa/data/method4/trank2010.html
https://kassiesa.net/uefa/data/method4/trank2011.html
https://kassiesa.net/uefa/data/method4/trank2012.html
https://kassiesa.net/uefa/data/method4/trank2013.html
https://kassiesa.net/uefa/data/method4/trank2014.html
https://kassiesa.net/uefa/data/method4/trank2015.html
https://kassiesa.net/uefa/data/method4/trank2016.html
https://kassiesa.net/uefa/data/method4/trank2017.html
https://kassiesa.net/uefa/data/method5/trank2018.html
https://kassiesa.net/uefa/da